In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
%reload_ext autoreload
%autoreload 2
%matplotlib inline

Imports

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np
import pandas as pd
import seaborn as sns

from postprocessor.core.multisignal.crosscorr import crosscorr, crosscorrParameters
from postprocessor.core.processes.findpeaks import findpeaks, findpeaksParameters
from scipy.stats import sem

# Load data

In [ ]:
data_dir = "../data/raw/"
group_name = "is20016_zwf1egf"

In [ ]:
filepath = data_dir + group_name
timeseries_filepath = filepath + "_flavin_timeseries.csv"
labels_filepath = filepath + "_labels.csv"

timeseries_df = pd.read_csv(timeseries_filepath, index_col=[0,1,2])
labels_df = pd.read_csv(labels_filepath, index_col=[0,1,2])

# Select data

Drop NaNs

In [ ]:
timeseries_dropna = timeseries_df.dropna()

In [ ]:
labels_dropna = labels_df.loc[timeseries_dropna.index]

Select oscillatory time series

In [ ]:
timeseries_osc = timeseries_dropna.loc[labels_dropna[labels_dropna.score == 1].index]

In [ ]:
timeseries_osc

# Estimate period

## Use ACF

Define convenience function

In [ ]:
def get_first_interval(x):
    """get interval lengths then get just the first one"""
    list_intervals = np.diff(np.where(np.array(x) > 0))[0]
    # Checks that it is not empty
    if list_intervals.any():
        return list_intervals[0]
    else:
        return np.nan

Define parameters for findpeaks

In [ ]:
prominence = 0.20
width = 4

Estimate periods

In [ ]:
acfs = crosscorr.as_function(timeseries_osc, normalised=True, only_pos=True)
acfs_peaks = findpeaks.as_function(
    acfs, prominence=prominence, width=width
)
periods = acfs_peaks.apply(lambda x: get_first_interval(x), axis=1)
periods_min = 5 * periods.to_numpy()

# Statistics

In [ ]:
num = timeseries_dropna.shape[0]
num_osc = len(periods_min)

mean = np.mean(periods_min)
std_err_mean = sem(periods_min)

median = np.median(periods_min)
q25, q75 = np.percentile(periods_min, [25,75])

print(f"n = {num}; n(osc) = {num_osc} ({100*num_osc/num:.2f}%).")
print(f"mean = {mean:.2f}; SEM = {std_err_mean:.2f}.")
print(f"median = {median:.2f}; IQR = {q25:.2f}--{q75:.2f} (diff = {q75-q25:.2f}).")